In [1]:
import json
import os
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient


def load_config(config_path='../config.json'):
    with open(config_path, 'r') as config_file:
        config = json.load(config_file)
    return config

config = load_config()
key = config['subscription_key']
endpoint = config['endpoint']

client = TextAnalyticsClient(endpoint=endpoint, credential=AzureKeyCredential(key))

#dictionar de cuvinte
emotion_keywords = {
    "bucurie": ["fericit", "incantat", "minunat", "bucuros", "vesel"],
    "entuziasm": ["entuziasmat", "superb", "fantastic", "uimitor"],
    "tristete": ["trist", "suparat", "deprimat", "nefericit"],
    "furie": ["furios", "enervat", "suparat", "revoltat"],
    "dezamagire": ["dezamagit", "prost", "ingrozitor", "deceptionat"],
    "neutralitate": ["obisnuit", "normal", "in regula", "fara"],
    "optimism" : ["posibil", "sper", "cred", "sigur", "sunt sigur"]
}

# lista  de mesaje de test
test_messages = [

    "Sunt absolut incantat de noul meu telefon, functioneaza perfect!",  # Entuziasm/Bucurie
    "Am avut o zi groaznica, totul a卓法 a mers prost la munca.",  # Dezamagire/Tristete
    "Maine este o zi obisnuita, fara planuri speciale.",  # Neutralitate
    "Mancarea la restaurant a fost delicioasa, iar servirea excelenta!",  # Bucurie
    "Am pierdut autobuzul si a trebuit sa astept o ora in ploaie.",  # Tristete/Dezamagire
    "Vremea este in regula, nici prea cald, nici prea frig.",  # Neutralitate
    "Sunt atat de recunoscator pentru sprijinul prietenilor mei!",  # Bucurie
    "Filmul pe care l-am vazut aseara a fost o pierdere de timp.",  # Dezamagire
    "Trebuie sa merg la magazin sa cumpar cateva lucruri.",  # Neutralitate
    "Calatoria in Italia a fost o experienta de neuitat!",  # Entuziasm/Bucurie
    "Sunt furios, mi-au anulat comanda fara sa ma anunte!",  # Furie
    "Ma simt deprimat, nu mai stiu ce sa fac cu viata mea.",  # Tristete
    "Am castigat la loto, ce zi fantastica!",  # Entuziasm/Bucurie

    "Am primit o promovare la munca, sunt super entuziasmat!",  # Entuziasm
    "Serviciul clienti a fost groaznic, m-au ignorat complet.",  # Furie/Dezamagire
    "Am terminat o carte interesanta, dar nu m-a impresionat prea mult.",  # Neutralitate/Dezamagire
    "Petrecerea de aseara a fost cea mai buna la care am fost vreodata!",  # Bucurie/Entuziasm
    "Mi-am pierdut portofelul si sunt foarte suparat.",  # Tristete
    "Trebuie sa-mi fac temele pentru maine, nimic special.",  # Neutralitate
    "Sunt revoltat de cat de scump a devenit totul in ultimul timp!",  # Furie

    "Am vazut un concert uimitor aseara, a fost incredibil!",  # Entuziasm
    "Trenul a intarziat iar, sunt atat de enervat!",  # Furie
    "Am avut o discutie placuta cu un coleg astazi.",  # Bucurie
    "Nu am dormit bine noaptea trecuta, ma simt obosit.",
    "By choosing a bike over a car, I’m reducing my environmental footprint. Cycling promotes eco-friendly transportation, and I’m proud to be part of that movement."# bucurie
]

def detect_emotion(text, sentiment, confidence_scores):
    if sentiment == "positive":
        possible_emotions = ["bucurie", "entuziasm","optimism"]
    elif sentiment == "negative":
        possible_emotions = ["tristete", "furie", "dezamagire"]
    else:
        possible_emotions = ["neutralitate"]

    text_lower = text.lower()
    detected_emotions = []
    for emotion, keywords in emotion_keywords.items():
        if any(keyword in text_lower for keyword in keywords):
            detected_emotions.append(emotion)

    if not detected_emotions:
        return possible_emotions[0]

    if len(detected_emotions) > 1:
        if sentiment == "positive" and "entuziasm" in detected_emotions:
            return "entuziasm"
        if sentiment == "negative" and "furie" in detected_emotions:
            return "furie"
        return detected_emotions[0]

    return detected_emotions[0]

def analyze_multiple_sentiments(texts, batch_size=10):
    try:
        if not texts:
            raise ValueError("Lista de texte este goala.")

        results = []
        # impart in bachuri propozitiile
        for i in range(0, len(texts), batch_size):
            batch = texts[i:i + batch_size]
            print(f"Procesez batch-ul {i//batch_size + 1} cu {len(batch)} mesaje...")
            response = client.analyze_sentiment(documents=batch, language="ro")

            for idx, doc in enumerate(response):
                if not doc.is_error:
                    emotion = detect_emotion(batch[idx], doc.sentiment, doc.confidence_scores)
                    results.append({
                        "text": batch[idx],
                        "sentiment": doc.sentiment,
                        "emotion": emotion,
                        "confidence_scores": {
                            "positive": doc.confidence_scores.positive,
                            "neutral": doc.confidence_scores.neutral,
                            "negative": doc.confidence_scores.negative
                        }
                    })
                else:
                    results.append({
                        "text": batch[idx],
                        "error": doc.error.message
                    })
        return results
    except Exception as e:
        print(f"Eroare la analiza sentimentului: {str(e)}")
        return {"error": str(e)}


try:
    results = analyze_multiple_sentiments(test_messages)
    if "error" in results:
        print(f"Eroare generala: {results['error']}")
    else:
        for result in results:
            print(json.dumps(result, indent=2, ensure_ascii=False))
except Exception as e:
    print(f"Eroare neasteptata: {str(e)}")

Procesez batch-ul 1 cu 10 mesaje...
Procesez batch-ul 2 cu 10 mesaje...
Procesez batch-ul 3 cu 5 mesaje...
{
  "text": "Sunt absolut incantat de noul meu telefon, functioneaza perfect!",
  "sentiment": "positive",
  "emotion": "bucurie",
  "confidence_scores": {
    "positive": 1.0,
    "neutral": 0.0,
    "negative": 0.0
  }
}
{
  "text": "Am avut o zi groaznica, totul a卓法 a mers prost la munca.",
  "sentiment": "negative",
  "emotion": "dezamagire",
  "confidence_scores": {
    "positive": 0.0,
    "neutral": 0.0,
    "negative": 1.0
  }
}
{
  "text": "Maine este o zi obisnuita, fara planuri speciale.",
  "sentiment": "neutral",
  "emotion": "neutralitate",
  "confidence_scores": {
    "positive": 0.01,
    "neutral": 0.67,
    "negative": 0.31
  }
}
{
  "text": "Mancarea la restaurant a fost delicioasa, iar servirea excelenta!",
  "sentiment": "positive",
  "emotion": "bucurie",
  "confidence_scores": {
    "positive": 1.0,
    "neutral": 0.0,
    "negative": 0.0
  }
}
{
  "text": "